# Data Analytics Project

## Purpose: Sales Prediction

The purpose of this project is to predict sales for a UK-based ecommerce business. 

This will be achieved through linear regression and support vector regression.

## 1: Module Importation and DataFrame Creation

In [244]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from matplotlib import pyplot as plt 
import sklearn
import warnings 
warnings.simplefilter('ignore', category=UserWarning)

%matplotlib inline

# The file containing the dataset uses a different encoding than the default 'utf-8', so this is specified

df = pd.read_csv('ecommerce.csv', encoding='cp1252') 

## 2: Data Inspection and Cleaning

The dataset being used needs to be cleaned before it can be explored and analysed. Using the head() function, an initial understanding of the dataset can be established.

In [245]:
print(f"There are {len(df.columns)} columns in the dataframe: {list(df.columns)}")
df.head() # returns the first five rows of the dataframe

There are 8 columns in the dataframe: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Everything looks alright, but the 'CustomerID' column seems to be registered as a float value. Most likely, this is because of null values which cannot be recorded for a variable of type 'int64'. The info() function can help confirm this, as it displays a list of data types and non-null value counts for each column in the dataframe.

In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


As suspected, the 'CustomerID' variable has a dtype of 'float64'. Using the astype() function, its type can be easily changed to 'object'. Before doing so, all null values will also be replaced to make plotting easier. In this case, the number 0 will work, because Customer IDs are typically five digits in length, so there will be no conflict.

In [247]:
df['CustomerID'] = df['CustomerID'].fillna(0.0)
df['CustomerID'] = df['CustomerID'].astype('int64')
df['CustomerID'] = df['CustomerID'].astype('object')
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID      object
Country         object
dtype: object

It's worth noting that the 'Description' variable still has 1454 rows with null values.

In [248]:
df.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

To make sure the dataset has no null values, the string 'NA' is used to fill the holes. The data is now clean!

In [249]:
df['Description'] = df['Description'].fillna('NA')
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

The describe() function can be used to get a general overview of basic statistics regarding the numerical variables. Notice that there are negative values present in the 'min' row. Further investigation is required.

In [250]:
df.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


After sorting the dataframe by 'Quantity' and returning the first 10 rows, it becomes obvious that some rows in the dataset are meant to be for inventory management purposes. That is, they have no bearing on the total sales. These rows should be removed.

In [262]:
df.sort_values(by = 'Quantity').head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,12/9/2011 9:27,2.08,16446,United Kingdom,-168469.60
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1/18/2011 10:17,1.04,12346,United Kingdom,-77183.60
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,12/2/2010 14:23,0.03,15838,United Kingdom,-280.80
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,4/18/2011 13:08,2.10,15749,United Kingdom,-6539.40
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,4/18/2011 13:08,1.85,15749,United Kingdom,-3700.00
160143,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,4/18/2011 13:08,2.55,15749,United Kingdom,-4921.50
224419,C556522,22920,HERB MARKER BASIL,-1515,6/13/2011 11:21,0.55,16938,United Kingdom,-833.25
187609,C552995,M,Manual,-1350,5/12/2011 15:19,0.16,18133,United Kingdom,-216.00
160142,C550456,47566B,TEA TIME PARTY BUNTING,-1300,4/18/2011 13:08,2.55,15749,United Kingdom,-3315.00
390548,C570556,20971,PINK BLUE FELT CRAFT TRINKET BOX,-1296,10/11/2011 11:10,1.06,16029,United Kingdom,-1373.76


In [252]:
for i in df.index:
    if df.loc[i, 'UnitPrice'] == 0.0:
        df.drop(i, inplace=True)

Because the goal is to predict the company's sales for the following year, having only 'Quantity' and 'UnitPrice' as variables does not paint the clearest picture of overall sales. Thankfully, the sale amount (or refund amount, in some cases) associated with each row in the dataframe can be easily computed using these two variables.

In [253]:
df['Total'] = (
    df['Quantity']
    * df['UnitPrice']
)
df.describe()

,Quantity,UnitPrice,Total
count,539394.000000,539394.000000,539394.000000
mean,9.845871,4.632614,18.071665
std,215.412253,96.984656,379.690931
min,-80995.000000,-11062.060000,-168469.600000
25%,1.000000,1.250000,3.750000
50%,3.000000,2.080000,9.840000
75%,10.000000,4.130000,17.400000
max,80995.000000,38970.000000,168469.600000


In [263]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680,France,16.60


### Categorical Variables

Moving on to categorical variables, these can also be summarized using the describe() function.

In [264]:
df.describe(include='object')

,InvoiceNo,StockCode,Description,InvoiceDate,CustomerID,Country
count,539394,539394,539394,539394,539394,539394
unique,23798,3938,4042,21804,4372,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,0,United Kingdom
freq,1114,2307,2365,1114,132605,492981
